# Mineração de Dados: Classificação

**Grupo**:

- Gabriel Oliveira Moreira Faria
- Vinícius Oliveira

**Objetivo**:

O objetivo deste trabalho é criar classificadores que buscam prever a aprovação (ou reprovação) de discentes que participaram de um curso online.
Essas bases de dados apresentam características de pessoas que participaram do curso online "Prevenção do uso de drogas".


Os dados estão disponíveis na pasta do drive compartilhada aqui e são formados por 3 arquivos:
"trabalho5_dados_sociais_ID.csv" - dados sócio-econômicos e de perguntas iniciais respondidas pelos participantes,
"trabalho5_dados_modulo1_ID.csv" - dados de acesso ao sistema de ensino durante as atividades referentes ao primeiro módulo do curso, e
"trabalho5_dados_ateh_modulo2_ID.csv" - dados de acesso ao sistema de ensino para as atividades até o segundo módulo do curso.


Nesse trabalho geramos um modelo para cada um das 3 situações a seguir:
1) um modelo que considera apenas as características sócio-econômicos e de perguntas iniciais;
2) um modelo que considera apenas as características sócio-econômicos e de perguntas iniciais, e de acesso considerando o primeiro módulo do curso; e
3) um modelo que considera todos os dados disponíveis.

Para cada uma das 3 situações apresentadas, deve-se gerar ao menos 2 modelos de tipos distintos.

**Com a importância de analisar os parâmetros que fornecemos para os modelos**

In [46]:
import itertools as it
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyclustertend
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import DBSCAN, OPTICS, AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestCentroid
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale

## Leitura dos dados e Tratamento dos dados

Primeiro, carregaremos os dados e analisaremos as características sócio-econômicas, perguntas e dados do sistem de ensino providos a partir da base de dados.


In [47]:
df_sociais = pd.read_csv("data/trabalho5_dados_sociais_4.csv")
df_sociais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 80 columns):
 #   Column                                                                          Non-Null Count  Dtype 
---  ------                                                                          --------------  ----- 
 0   id                                                                              1000 non-null   int64 
 1   idade                                                                           1000 non-null   int64 
 2   sexo                                                                            1000 non-null   object
 3   escolaridade                                                                    1000 non-null   object
 4   estadocivil                                                                     1000 non-null   object
 5   ocupacao                                                                        1000 non-null   object
 6   tempodeservico           

In [48]:
df_sociais

,id,idade,sexo,escolaridade,estadocivil,ocupacao,tempodeservico,religiao,contatoanterior,lidadiretamente,...,Ausência de regras no ambiente escolar (barreiras),Possuir alunos interessados na temática (facilitadores),Presença de uma equipe para trabalhar a temática (facilitadores),Estímulo aos alunos (facilitadores),Desenvolvimento de projetos na escola (facilitadores),Apoio aos projetos em desenvolvimento (facilitadores),Presença de regras no ambiente escolar (facilitadores),Promoção de compromisso e confiança (facilitadores),Valorização do ambiente escolar (facilitadores),Participação da comunidade e dos pais no trabalho de prevenção (facilitadores)
0,7,45,Feminino,Pós-graduação,Casado (a),Professor (a),22,Católica,Sim,Sim,...,0,1,1,1,1,0,0,1,1,1
1,33,49,Masculino,Pós-graduação,Casado (a),Outros,29,Católica,Não,Sim,...,1,1,1,1,1,1,1,1,1,1
2,45,49,Feminino,Pós-graduação,Divorciado (a),Professor (a),19,Católica,Sim,Sim,...,0,1,0,0,0,0,0,0,1,1
3,50,55,Masculino,Pós-graduação,Casado (a),Professor (a),29,Católica,Não,Não,...,0,0,0,0,0,0,0,0,0,0
4,58,49,Masculino,Ensino Superior Completo,Casado (a),Professor (a),10,Católica,Sim,Sim,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10859,47,Feminino,Pós-graduação,Casado (a),Professor (a),25,Católica,Sim,Sim,...,0,0,0,0,0,0,0,0,0,0
996,10872,42,Feminino,Pós-graduação,Casado (a),Professor (a),20,Católica,Sim,Sim,...,0,1,1,0,1,1,0,1,1,0
997,10873,46,Feminino,Ensino Superior Completo,Casado (a),Professor (a),4,Católica,Sim,Não,...,0,0,0,0,0,0,0,0,0,0
998,10876,48,Feminino,Pós-graduação,Casado (a),Professor (a),16,Católica,Não,Sim,...,0,1,1,0,1,1,0,0,0,1


In [49]:
df_sociais.iloc[:,0:18]

,id,idade,sexo,escolaridade,estadocivil,ocupacao,tempodeservico,religiao,contatoanterior,lidadiretamente,lida.onde,materialdidatico,prazoatividades,interacaopares,organizacaocurso,import.ajud.tutor,autoavaliacao.x,part.outrocurso
0,7,45,Feminino,Pós-graduação,Casado (a),Professor (a),22,Católica,Sim,Sim,Escola,Adequado,Flexível,Importante,Organizado,Sempre,"Sim, considero, porém, poderia estar me esforç...",Sim
1,33,49,Masculino,Pós-graduação,Casado (a),Outros,29,Católica,Não,Sim,Escola,Muito adequado,Flexível,Importante,Muito organizado,Sempre,"Sim, considero, porém, poderia estar me esforç...",Não
2,45,49,Feminino,Pós-graduação,Divorciado (a),Professor (a),19,Católica,Sim,Sim,Escola,Muito adequado,Muito flexível,Importante,Muito organizado,Sempre,"Sim, considero",Sim
3,50,55,Masculino,Pós-graduação,Casado (a),Professor (a),29,Católica,Não,Não,Escola,Adequado,Flexível,Importante,Organizado,Sempre,"Sim, considero, porém, poderia estar me esforç...",Não
4,58,49,Masculino,Ensino Superior Completo,Casado (a),Professor (a),10,Católica,Sim,Sim,Amigos,Adequado,Pouco flexível,Importante,Organizado,Sempre,"Sim, considero, porém, poderia estar me esforç...",Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10859,47,Feminino,Pós-graduação,Casado (a),Professor (a),25,Católica,Sim,Sim,Escola,Adequado,Flexível,Importante,Organizado,Às vezes,"Sim, considero",Não
996,10872,42,Feminino,Pós-graduação,Casado (a),Professor (a),20,Católica,Sim,Sim,Escola,Muito adequado,Flexível,Importante,Muito organizado,Sempre,"Sim, considero",Não
997,10873,46,Feminino,Ensino Superior Completo,Casado (a),Professor (a),4,Católica,Sim,Não,Família,Adequado,Flexível,Importante,Organizado,Sempre,"Sim, considero",Sim
998,10876,48,Feminino,Pós-graduação,Casado (a),Professor (a),16,Católica,Não,Sim,Comunidade,Muito adequado,Flexível,Muito importante,Organizado,Sempre,"Sim, considero, porém, poderia estar me esforç...",Não


In [50]:
print(df_sociais.materialdidatico.unique())

df_sociais.materialdidatico = df_sociais.materialdidatico.replace({'Adequado': 1, 'Muito adequado': 2})

print(df_sociais.materialdidatico.unique())

['Adequado' 'Muito adequado']
[1 2]


In [51]:
print(df_sociais.prazoatividades.unique())

df_sociais.prazoatividades = df_sociais.prazoatividades.replace({'Pouquíssimo flexível': 1, 'Pouco flexível': 2, 'Flexível': 3, 'Muito flexível': 4})

print(df_sociais.prazoatividades.unique())

['Flexível' 'Muito flexível' 'Pouco flexível' 'Pouquíssimo flexível']
[3 4 2 1]


In [52]:
print(df_sociais.interacaopares.unique())

df_sociais.interacaopares = df_sociais.interacaopares.replace({'Importante': 1, 'Muito importante': 2})

print(df_sociais.interacaopares.unique())

['Importante' 'Muito importante']
[1 2]


In [53]:
print(df_sociais.organizacaocurso.unique())

df_sociais.organizacaocurso = df_sociais.organizacaocurso.replace({'Organizado': 1, 'Muito organizado': 2})

print(df_sociais.organizacaocurso.unique())

['Organizado' 'Muito organizado']
[1 2]


In [54]:
print(df_sociais['import.ajud.tutor'].unique())

df_sociais['import.ajud.tutor'] = df_sociais['import.ajud.tutor'].replace({'Às vezes': 1, 'Sempre': 2})

print(df_sociais['import.ajud.tutor'].unique())

['Sempre' 'Às vezes']
[2 1]


In [55]:
print(df_sociais['autoavaliacao.x'].unique())

df_sociais['autoavaliacao.x'] = df_sociais['autoavaliacao.x'].replace({'Não, não considero': 1, 'Sim, considero, porém, poderia estar me esforçando mais': 2, 'Sim, considero': 3})

print(df_sociais['autoavaliacao.x'].unique())


['Sim, considero, porém, poderia estar me esforçando mais'
 'Sim, considero' 'Não, não considero']
[2 3 1]


In [56]:
df_sociais['lida.onde']

0          Escola
1          Escola
2          Escola
3          Escola
4          Amigos
          ...    
995        Escola
996        Escola
997       Família
998    Comunidade
999        Escola
Name: lida.onde, Length: 1000, dtype: object

In [11]:
df_sociais.columns

Index(['id', 'idade', 'sexo', 'escolaridade', 'estadocivil', 'ocupacao',
       'tempodeservico', 'religiao', 'contatoanterior', 'lidadiretamente',
       'lida.onde', 'materialdidatico', 'prazoatividades', 'interacaopares',
       'organizacaocurso', 'import.ajud.tutor', 'autoavaliacao.x',
       'part.outrocurso', 'pp001', 'pp002', 'pp003', 'pp004', 'pp005', 'pp006',
       'pp007', 'pp008', 'pp009', 'pp010', 'pp011', 'pp012', 'pp013', 'pp014',
       'pp015', 'pp016', 'pp017', 'pp018', 'pp019', 'pp020', 'pp021', 'pp022',
       'pp023', 'pp024', 'pp025', 'pp026', 'pp027', 'pp028', 'pp029', 'pp030',
       'pp031', 'pp032', 'pp033', 'pp034', 'pp035', 'pp036', 'pp037',
       'aprovado', 'Identificação pessoal com o tema (motivopart)',
       'Identificação profissional com o tema (motivopart)',
       'Para aquisição de conhecimento na área (motivopart)',
       'Pelo fato de o curso ser gratuito (motivopart)',
       'Pelo fato de o curso estar vinculado à Universidade (motivopart)'

Não existe campos nulos

In [21]:
df_sociais.isna().sum().sum()

0

#### Substituição das variáveis qualitativas

- O que é "Sim" para 1 e do que é "Não" para 0

In [15]:
df_sociais = df_sociais.replace('Sim', 1).replace('Não', 0)

In [16]:
df_sociais

,id,idade,sexo,escolaridade,estadocivil,ocupacao,tempodeservico,religiao,contatoanterior,lidadiretamente,...,Ausência de regras no ambiente escolar (barreiras),Possuir alunos interessados na temática (facilitadores),Presença de uma equipe para trabalhar a temática (facilitadores),Estímulo aos alunos (facilitadores),Desenvolvimento de projetos na escola (facilitadores),Apoio aos projetos em desenvolvimento (facilitadores),Presença de regras no ambiente escolar (facilitadores),Promoção de compromisso e confiança (facilitadores),Valorização do ambiente escolar (facilitadores),Participação da comunidade e dos pais no trabalho de prevenção (facilitadores)
0,7,45,Feminino,Pós-graduação,Casado (a),Professor (a),22,Católica,1,1,...,0,1,1,1,1,0,0,1,1,1
1,33,49,Masculino,Pós-graduação,Casado (a),Outros,29,Católica,0,1,...,1,1,1,1,1,1,1,1,1,1
2,45,49,Feminino,Pós-graduação,Divorciado (a),Professor (a),19,Católica,1,1,...,0,1,0,0,0,0,0,0,1,1
3,50,55,Masculino,Pós-graduação,Casado (a),Professor (a),29,Católica,0,0,...,0,0,0,0,0,0,0,0,0,0
4,58,49,Masculino,Ensino Superior Completo,Casado (a),Professor (a),10,Católica,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10859,47,Feminino,Pós-graduação,Casado (a),Professor (a),25,Católica,1,1,...,0,0,0,0,0,0,0,0,0,0
996,10872,42,Feminino,Pós-graduação,Casado (a),Professor (a),20,Católica,1,1,...,0,1,1,0,1,1,0,1,1,0
997,10873,46,Feminino,Ensino Superior Completo,Casado (a),Professor (a),4,Católica,1,0,...,0,0,0,0,0,0,0,0,0,0
998,10876,48,Feminino,Pós-graduação,Casado (a),Professor (a),16,Católica,0,1,...,0,1,1,0,1,1,0,0,0,1


In [13]:
df_sociais.aprovado.unique()

array([1, 0], dtype=int64)

In [4]:
df_modulo1 = pd.read_csv("data/trabalho5_dados_modulo1_4.csv")
df_modulo1.info()

# não existe dados nulos, todas linhas de todas as colunas estão preenchidas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1000 non-null   int64 
 1   quesm1      1000 non-null   int64 
 2   quesm1r     1000 non-null   int64 
 3   forum1      1000 non-null   int64 
 4   forum2      1000 non-null   int64 
 5   forum3      1000 non-null   int64 
 6   forum4      1000 non-null   int64 
 7   ativcolm1   1000 non-null   int64 
 8   ativcolm1r  1000 non-null   int64 
 9   forum1r     1000 non-null   int64 
 10  aprovado    1000 non-null   object
dtypes: int64(10), object(1)
memory usage: 86.1+ KB


In [5]:
df_modulo2 = pd.read_csv("data/trabalho5_dados_ateh_modulo2_4.csv")
df_modulo2.info()

# não existe dados nulos, todas linhas de todas as colunas estão preenchidas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1000 non-null   int64 
 1   quesm2      1000 non-null   int64 
 2   ativcolm2   1000 non-null   int64 
 3   forum5      1000 non-null   int64 
 4   forum6      1000 non-null   int64 
 5   forum7      1000 non-null   int64 
 6   ativcolm2r  1000 non-null   int64 
 7   quesm2r     1000 non-null   int64 
 8   forum8      1000 non-null   int64 
 9   forum2r     1000 non-null   int64 
 10  aprovado    1000 non-null   object
 11  quesm1      1000 non-null   int64 
 12  quesm1r     1000 non-null   int64 
 13  forum1      1000 non-null   int64 
 14  forum2      1000 non-null   int64 
 15  forum3      1000 non-null   int64 
 16  forum4      1000 non-null   int64 
 17  ativcolm1   1000 non-null   int64 
 18  ativcolm1r  1000 non-null   int64 
 19  forum1r     1000 non-null   int64 
dtypes: int64(

# Análises exploratórias, entendimento e tratamento dos dados

## Situação 1 - Características sócio-econômicas e perguntas iniciais

# Situação 2 - Características sócio-econômicas, perguntas iniciais e primeiro módulo do curso

# Situação 2 - Características sócio-econômicas, perguntas iniciais, primeiro e segundo módulo do curso (todos os dados)